In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
model_name = 'nasnet_8classifier_trained'

train_batch_size=8
import keras.backend as K
import tensorflow as tf

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

Using TensorFlow backend.


# 导入必要的库

In [2]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model

from IPython.display import display, Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 读取数据集

In [3]:
df = pd.read_csv('../train/Annotations/train.csv', header=None)
df.columns = ['filename', 'label_name', 'label']
df = df.sample(frac=1).reset_index(drop=True) # shuffle

df.label_name = df.label_name.str.replace('_labels', '')

display(df.head())
c = Counter(df.label_name)
c

,filename,label_name,label
0,Images/collar_design_labels/9a015b42c4d2a2f4e4...,collar_design,nnnyn
1,Images/neckline_design_labels/a4cb0151431b18b8...,neckline_design,nynnnnnnnn
2,Images/skirt_length_labels/a0d69afce8845f87641...,skirt_length,nnnnyn
3,Images/sleeve_length_labels/50330ed94647da3524...,sleeve_length,nnnnnnnny
4,Images/skirt_length_labels/4302bd7414e68b44c42...,skirt_length,nnnnny


Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [4]:
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
display(label_count)

{'coat_length': 8,
 'collar_design': 5,
 'lapel_design': 5,
 'neck_design': 5,
 'neckline_design': 10,
 'pant_length': 6,
 'skirt_length': 6,
 'sleeve_length': 9}

## 生成 y

In [5]:
fnames = df['filename'].values
width = 331
#width = 299
n = len(df)
y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1

## 读取图片

In [6]:
def f(index):
    return index, cv2.resize(cv2.imread('../train/'+fnames[index]), (width, width))

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(30) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

100%|██████████| 89683/89683 [05:00<00:00, 298.78it/s]


In [7]:
n_train = int(n*0.83)

X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

In [8]:
def display_images(imgs, w=8, h=4, figsize=(24, 12)):
    plt.figure(figsize=figsize)
    for i in range(w*h):
        plt.subplot(h, w, i+1)
        plt.imshow(imgs[i])

In [9]:
class Generator():
    def __init__(self, X, y, batch_size=32, aug=False):
        def generator():
            idg = ImageDataGenerator(horizontal_flip=True,
                                     rotation_range=20,
                                     zoom_range=0.15)
            while True:
                for i in range(0, len(X), batch_size):
                    X_batch = X[i:i+batch_size].copy()
                    y_barch = [x[i:i+batch_size] for x in y]
                    if aug:
                        for j in range(len(X_batch)):
                            X_batch[j] = idg.random_transform(X_batch[j])
                    yield X_batch, y_barch
        self.generator = generator()
        self.steps = len(X) // batch_size + 1

In [10]:
gen_train = Generator(X_train, y_train, batch_size=train_batch_size, aug=True)

# 数据集探索

## 类别分布

In [11]:
# plt.figure(figsize=(26, 14))
# for i in range(8):
#     plt.subplot(2, 4, i+1)
#     counts = Counter(y[i].argmax(axis=-1)[np.where(y[i].any(axis=-1))])
#     pd.Series(counts).plot('bar')

### 抽样可视化

In [12]:
# plt.figure(figsize=(26, 14))
# w = 8
# h = 4
# for i in range(w*h):
#     plt.subplot(h, w, i+1)
#     index = np.random.randint(n)
#     plt.title(str([y[x][index].argmax() if y[x][index].any() else -1 for x in range(8) ]))
#     plt.imshow(X[index])
#     plt.axis('off')

# 搭建模型并训练

## 搭建模型

In [13]:
def acc(y_true, y_pred):
    index = tf.reduce_any(y_true > 0.5, axis=-1)
    res = tf.equal(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1))
    index = tf.cast(index, tf.float32)
    res = tf.cast(res, tf.float32)
    return tf.reduce_sum(res * index) / (tf.reduce_sum(index) + 1e-7)

In [14]:
base_model = NASNetLarge(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling='avg')
#InceptionResNetV2,Xception
input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(nasnet.preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot, plot_model

#plot_model(model, show_shapes=True, to_file='model_simple.png')
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [15]:
#model2 = multi_gpu_model(model, n_gpus)
model2 = model

In [16]:
# opt = SGD(1e-3, momentum=0.9, nesterov=True, decay=1e-5)

In [17]:
model2.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2
9305/9305 [==============================] - 8554s 919ms/step - loss: 0.8700 - collar_design_loss: 0.0852 - neckline_design_loss: 0.2050 - skirt_length_loss: 0.1362 - sleeve_length_loss: 0.1333 - lapel_design_loss: 0.0681 - coat_length_loss: 0.1132 - pant_length_loss: 0.0587 - neck_design_loss: 0.0704 - collar_design_acc: 0.3564 - neckline_design_acc: 0.5237 - skirt_length_acc: 0.6605 - sleeve_length_acc: 0.4878 - lapel_design_acc: 0.3219 - coat_length_acc: 0.4429 - pant_length_acc: 0.3667 - neck_design_acc: 0.2304 - val_loss: 0.6785 - val_collar_design_loss: 0.0703 - val_neckline_design_loss: 0.1485 - val_skirt_length_loss: 0.0984 - val_sleeve_length_loss: 0.1064 - val_lapel_design_loss: 0.0580 - val_coat_length_loss: 0.0888 - val_pant_length_loss: 0.0419 - val_neck_design_loss: 0.0662 - val_collar_design_acc: 0.2380 - val_neckline_design_acc: 0.4236 - val_skirt_length_acc: 0.5218 - val_sleeve_length_acc: 0.3575 - val_lapel_design_acc: 0.2138 - val_coat_length_acc: 0.2984 - 

In [18]:
model2.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3
9305/9305 [==============================] - 8416s 904ms/step - loss: 0.3224 - collar_design_loss: 0.0336 - neckline_design_loss: 0.0747 - skirt_length_loss: 0.0493 - sleeve_length_loss: 0.0446 - lapel_design_loss: 0.0225 - coat_length_loss: 0.0443 - pant_length_loss: 0.0229 - neck_design_loss: 0.0306 - collar_design_acc: 0.4750 - neckline_design_acc: 0.7119 - skirt_length_acc: 0.7857 - sleeve_length_acc: 0.6512 - lapel_design_acc: 0.4306 - coat_length_acc: 0.5788 - pant_length_acc: 0.4475 - neck_design_acc: 0.3423 - val_loss: 0.3672 - val_collar_design_loss: 0.0392 - val_neckline_design_loss: 0.0925 - val_skirt_length_loss: 0.0494 - val_sleeve_length_loss: 0.0494 - val_lapel_design_loss: 0.0265 - val_coat_length_loss: 0.0515 - val_pant_length_loss: 0.0257 - val_neck_design_loss: 0.0331 - val_collar_design_acc: 0.4646 - val_neckline_design_acc: 0.6968 - val_skirt_length_acc: 0.7811 - val_sleeve_length_acc: 0.6368 - val_lapel_design_acc: 0.4457 - val_coat_length_acc: 0.5721 - 

In [19]:
model2.compile(optimizer=Adam(1e-6), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

Epoch 1/1
9305/9305 [==============================] - 8792s 945ms/step - loss: 0.1758 - collar_design_loss: 0.0198 - neckline_design_loss: 0.0436 - skirt_length_loss: 0.0250 - sleeve_length_loss: 0.0234 - lapel_design_loss: 0.0100 - coat_length_loss: 0.0226 - pant_length_loss: 0.0130 - neck_design_loss: 0.0185 - collar_design_acc: 0.5039 - neckline_design_acc: 0.7552 - skirt_length_acc: 0.8169 - sleeve_length_acc: 0.6898 - lapel_design_acc: 0.4608 - coat_length_acc: 0.6202 - pant_length_acc: 0.4711 - neck_design_acc: 0.3699 - val_loss: 0.3524 - val_collar_design_loss: 0.0376 - val_neckline_design_loss: 0.0880 - val_skirt_length_loss: 0.0393 - val_sleeve_length_loss: 0.0515 - val_lapel_design_loss: 0.0217 - val_coat_length_loss: 0.0561 - val_pant_length_loss: 0.0249 - val_neck_design_loss: 0.0334 - val_collar_design_acc: 0.4708 - val_neckline_design_acc: 0.7048 - val_skirt_length_acc: 0.7948 - val_sleeve_length_acc: 0.6466 - val_lapel_design_acc: 0.4564 - val_coat_length_acc: 0.5763 - 

# 计算验证集准确率

In [ ]:
#model.save('B_model_%s.h5' % model_name)

In [31]:
y_pred = model2.predict(X_valid, batch_size=128, verbose=1)
a = np.array([x.any(axis=-1) for x in y_valid]).T.astype('uint8')
b = [np.where((a == np.eye(8)[x]).all(axis=-1))[0] for x in range(8)]
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    print(label_names[c], (y_pred2 == y_true2).mean())

15247/15247 [==============================] - 388s 25ms/step
collar_design 0.863572433193
neckline_design 0.869475847893
skirt_length 0.935757575758
sleeve_length 0.886933567972
lapel_design 0.921375921376
coat_length 0.870047543582
pant_length 0.89477786438
neck_design 0.833333333333


In [23]:
counts = Counter(df.label_name)
counts

Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [24]:
s = 0
n = 0
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    s += counts[label_names[c]] * (y_pred2 == y_true2).mean()
    n += counts[label_names[c]]
print(s / n)

0.889752483339


# 在测试集上预测

In [25]:
df_test = pd.read_csv('../test/z_rank/Tests/question.csv', header=None)
df_test.columns = ['filename', 'label_name', 'label']

fnames_test = df_test.filename

n_test = len(df_test)
df_test.head()

,filename,label_name,label
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels,?
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,?
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels,?
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels,?
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels,?


In [28]:
def f(index):
    return index, cv2.resize(cv2.imread('../test/z_rank/'+fnames_test[index]), (width, width))

X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n_test)), total=n_test) as pbar:
        for i, img in pbar:
            X_test[i] = img[:,:,::-1]

100%|██████████| 15042/15042 [01:19<00:00, 189.15it/s]


In [29]:
y_pred = model2.predict(X_test, batch_size=128, verbose=1)

15042/15042 [==============================] - 397s 26ms/step


In [30]:
for i in range(n_test):
    problem_name = df_test.label_name[i].replace('_labels', '')
    problem_index = label_names.index(problem_name)
    probs = y_pred[problem_index][i]
    df_test.label[i] = ';'.join(np.char.mod('%.8f', probs))

In [32]:
fname_csv = 'B_test_%s.csv' % model_name
df_test.to_csv(fname_csv, index=None, header=None)